In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://posticous-regine-deliberatively.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://posticous-regine-deliberatively.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），正式全名為明新學校財團法人明新科技大學，是一所位於台灣新竹縣新豐鄉的私立科技大學。

**歷史沿革**
明新科技大學創立於1966年（民國55年），最初以「明新工業專科學校」立案，設有機械、土木、工業管理等科別，並首次招收300名新生。 1993年，更名為「私立明新工商專科學校」。1997年7月，經教育部核准改制為「明新技術學院」並附設專科部。 最終在2002年8月，獲教育部核准優先升格為「明新科技大學」。2018年12月，更名為「明新學校財團法人明新科技大學」。

**學術組織與願景**
目前，明新科技大學設有半導體學院、工程學院、管理學院、民生學院（或稱服務產業學院）、人文與設計學院、共同教育學院等六大學院。學校積極配合國家經濟發展，致力於培育具備專業技能、團隊精神、人文素養與宏觀服務的科技人才。其願景為「深耕在地、放眼國際」，並以「培養具實務經驗與人文素養之專業人才」為目標。校名「明新」取自《大學》「在明明德，在新民，在止於至善」之精義，旨在闡揚人類與生俱來之德性與情操，期盼學子能涵養高尚品德，擁有專業學問與優良技術，達到全人發展的境界，並淬鍊「堅毅、求新、創造」的校訓精神。

**學校特色**
明新科技大學地處新竹縣新豐鄉，佔地逾30公頃，交通便利，鄰近省道縱貫線與中山高速公路。學校與時俱進，密切掌握世界與產業趨勢，致力於創新與優化學校方針，定位為「一流產業大學」。特別值得一提的是，明新科大積極與產業接軌，例如：
*   **半導體產業合作**：鑑於學校位於桃竹苗產業中心，鄰近科學園區與工業區，明新科大攜手國際封測大廠，成立「半導體封裝測試產業聯盟」，為半導體產業聚落提供優質人才。2021年，更整合電機工程、電子工程、化學工程與材料科技、光電工程等系，成立半導體學院。2022年10月，獲教育部核准通過「半導體科技博士學位學程」，為該校第一個博士班。
*   **AI與智慧科技**：為迎接AI時代，學校成立「AI教育推動中心」與「AI產學推動中心」，建置AI應用與創客教室，並將程式設計與人工智慧課程列為大一必修學分，透過導入AI與AR的「雙A」情境學習，培養學生的數位資訊素養。學校也建置了「永續智慧商務」教學與實習場域，發展「智慧零售」、「智慧金融」、「

In [9]:
result2 = stateful_query("校長是誰？")
print(result2)

None


In [10]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
